# IR PROJECT CODE

Members:

*   Omer
*   Lucas
*   Muhamad

# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

GEMMA3 270M SETUP

Based on: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=pCqnaKmlO1U9

In [1]:
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install pyterrier
!pip install pyterrier[java]
!pip install gensim
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install nltk
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from unsloth import FastLanguageModel
import torch
import torch.nn.functional as F
import pyterrier as pt
from pyterrier.measures import P, MAP, R, nDCG
from pyterrier_t5 import MonoT5ReRanker
import os
import json
import pandas as pd
import shutil
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import numpy as np
import pickle
from gensim.models import KeyedVectors

import csv
import xml.etree.ElementTree as ET
from xml.dom import minidom

  Cloning https://github.com/terrierteam/pyterrier_t5.git to /tmp/pip-req-build-3kl07mup
  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_t5.git /tmp/pip-req-build-3kl07mup
  Resolved https://github.com/terrierteam/pyterrier_t5.git to commit 81f4c49c6541e6ceedffd2e705cf2fe20089c3ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 114.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to ma

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o validation.json

import json

files = ["train.json", "validation.json", "test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1079  100  1079    0     0   3572      0 --:--:-- --:--:-- --:--:--  3584
100 71.5M  100 71.5M    0     0  37.9M      0  0:00:01  0:00:01 --:--:-- 51.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1079  100  1079    0     0   2711      0 --:--:-- --:--:-- --:--:--  2717
100 1315M  100 1315M    0     0  68.8M      0  0:00:19  0:00:19 --:--:-- 79.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1073  100  1073    0     0   2697      0 --:--:-- --:--:-- --:--:--  2695
100 71.8M  100 71.8M    0     0  13.9M      0  0:00:05  0:00:05 --:--:-- 20.3M

===== train.json =====
Preview of first 500 charact

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [2]:
import json
import os

inputs = ["train.json", "validation.json", "test.json"]
output = "document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from train.json
Loaded 24111 records from validation.json
Loaded 24084 records from test.json
Wrote 131921 records to document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood thereon, Your blood again

## You should check that the collection you have matches that of the paper!

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [3]:
import json
import re
import unicodedata
import string

input_file = "test.json"
output_file = "test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [4]:
input_file = "test.json"
qrels_file = "test_qrels.json"
answers_file = "test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to test_qrels.json
Saved 24084 entries to test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval - Good Luck! (YOUR WORK STARTS HERE !)

# Create the Index of the collection (2 min)



In [ ]:
#path index folder
folder_name = "Index"
index_file_name = "terrier_inverted_index"
index_path = os.path.abspath(os.path.join(folder_name, index_file_name))

#init pyTer
if not pt.java.started():
    pt.java.init()

def get_index():
    #check if the index exsists
    properties_file = os.path.join(index_path, "data.properties")
    # Forcing a rebuild by removing the index directory if it exists to ensure updated schema.
    if os.path.exists(index_path):
        print(f"Existing index found at {index_path}. Removing to ensure rebuild with updated schema.")
        shutil.rmtree(index_path)

    print("Index is not found or is being rebuilt, creating a new Index")
    os.makedirs(index_path, exist_ok=True)

    with open('document_collection.json', 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    # Build Data Frame to include docno, context, raw_ocr, and publication_date
    df = pd.DataFrame(raw_data)
    df = df.rename(columns={"para_id": "docno"})[["docno", "context", "raw_ocr", "publication_date"]]

    df['text'] = df['context'] # Create a 'text' column for the primary indexed field

    # Build the index
    indexer = pt.IterDictIndexer(
        index_path,
        meta={"docno": 24, "context": 2048, "raw_ocr": 2048, "text": 2048},
        text_attrs=["text"],           # 'text' (our 'context') is the primary field for indexing content
        meta_reverse=["docno"],        # enable reverse lookup on docno
        pretokenised=False,
        fields=False,
        threads=1,
    )
    index_ref = indexer.index(df.to_dict(orient="records"))
    return pt.IndexFactory.of(index_ref)
index = get_index()

# Print a simple summary
print("Index location:", index_path)
print("Indexed documents:", index.getCollectionStatistics().getNumberOfDocuments())

In [6]:
# Retrieve collection statistics
stats = index.getCollectionStatistics()

print("Terrier Collection Statistics")
print("--------------------------------")
print(f"Indexed documents:        {stats.getNumberOfDocuments()}")
print(f"Unique terms (vocabulary): {stats.getNumberOfUniqueTerms()}")
print(f"Total tokens:             {stats.getNumberOfTokens()}")
print(f"Average document length:  {stats.getAverageDocumentLength():.2f}")

Terrier Collection Statistics
--------------------------------
Indexed documents:        131921
Unique terms (vocabulary): 236646
Total tokens:             15575099
Average document length:  118.06


# Set up query expansion with histwords (2 mins)

In [7]:
!wget http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
!unzip -n eng-fiction-all_sgns.zip
!ls -F

--2026-01-13 19:39:21--  http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400120997 (382M) [application/zip]
Saving to: ‘eng-fiction-all_sgns.zip.1’

eng-fiction-all_sgn 100%[===================>] 381.58M  40.1MB/s    in 13s     

2026-01-13 19:39:35 (28.5 MB/s) - ‘eng-fiction-all_sgns.zip.1’ saved [400120997/400120997]

Archive:  eng-fiction-all_sgns.zip
document_collection.json    sgns/		     train.json
eng-fiction-all_sgns.zip    test.json		     unsloth_compiled_cache/
eng-fiction-all_sgns.zip.1  test_qrels.json	     validation.json
Index/			    test_queries.json
sample_data/		    test_query_answers.json


In [8]:
def load_histwords_colab(year):
    path_vocab = f"sgns/{year}-vocab.pkl"
    path_vectors = f"sgns/{year}-w.npy"

    print(f"Loading {year} data...")

    with open(path_vocab, 'rb') as f:
        vocab = pickle.load(f)

    vectors = np.load(path_vectors)

    kv = KeyedVectors(vector_size=vectors.shape[1])
    kv.add_vectors(vocab, vectors)

    print(f"Success! Model for {year} loaded with {len(vocab)} words.")
    return kv

hist_model = load_histwords_colab(1850)

Loading 1850 data...
Success! Model for 1850 loaded with 100000 words.


In [ ]:
word = "gay"
if word in hist_model:
    print(f"Synonyms in 1850 for '{word}':")
    print(hist_model.most_similar(word, topn=3))

Synonyms in 1850 for 'gay':
[('brilliant', 0.7579882144927979), ('lively', 0.7548628449440002), ('graceful', 0.6951406598091125)]


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


### Setup the pipeline before 1st stage retrieval (Query expansion -> Query reduction)

In [9]:
def hist_expansion_logic(row):
    terms = row['query'].split()
    expanded_query = list(terms)

    for term in terms:
        word = term.lower()

        if word in hist_model.key_to_index:
            word_index = hist_model.key_to_index[word]
            if np.isclose(hist_model.get_vector(word, norm=False).tolist(), 0).all(): # Check if vector is all zeros
                continue

            try:
                sim_word, _ = hist_model.most_similar(word, topn=1)[0]
                sim_low = sim_word.lower()
            except Exception:
                continue

    return " ".join(expanded_query)

hist_query_expansion = pt.apply.query(hist_expansion_logic)

In [10]:
def query_reduction_by_weight(row):
    original_query = row['query']
    terms = original_query.split()
    reduced_terms = []

    for term in terms:
        if term.lower() not in stop_words:
            reduced_terms.append(term)
    return " ".join(reduced_terms) if reduced_terms else original_query

query_reduction = pt.apply.query(query_reduction_by_weight)

example_query = "How many lots did Thomas Peirce have in the year 1800"
reduced_query = query_reduction.transform(pd.DataFrame([{'qid': 'test_q', 'query': example_query}]))

print(f"Original request: {example_query}")
print(f"Reduced request: {reduced_query['query'].iloc[0]}")

Original request: How many lots did Thomas Peirce have in the year 1800
Reduced request: many lots Thomas Peirce year 1800


## Defining the query pipeline with a test

In [14]:
# DEFINE IT TO USE IT FOR OUR CUSTOM IR SYSTEM
query_pipeline = hist_query_expansion >> query_reduction


test_query = "Who lives in Europe with Alex and what is an apple ?"
df_test = pd.DataFrame([["q1", test_query]], columns=['qid', 'query'])
transformed_df = query_pipeline.transform(df_test)

print("INPUT:", test_query)
print("OUTPUT:", transformed_df.iloc[0]['query'])

INPUT: Who lives in Europe with Alex and what is an apple ?
OUTPUT: lives Europe Alex apple ?


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


# 1st stage retrieval

In [15]:
# First stage retriver with BM25
bm25_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="BM25")%5
tf_idf_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="TF_IDF")%5
bm25_custom_1st_stage_retrieval = query_pipeline >> bm25_1st_stage_retrieval

In [ ]:
#testing the engine
query = "Europe"
bm25_results = bm25_1st_stage_retrieval.search(query)
tf_idf_results = tf_idf_1st_stage_retrieval.search(query)
bm25_custom_results = bm25_custom_1st_stage_retrieval.search(query)
#show first 10 results
print('-------------------- bm25 results (top 5) --------------------\n',bm25_results.head(5))
print('-------------------- tf_idf results (top 5) --------------------\n',tf_idf_results.head(5))
print('-------------------- bm25_custom results (top 5) --------------------\n',bm25_custom_results.head(5))

# WARNING taking the whole queries will take a long time
# --- Use a subset of queries for faster computation,  ---
num_queries_subset = 30 # You can adjust this number

queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results = pt.Experiment(
    [bm25_1st_stage_retrieval, tf_idf_1st_stage_retrieval, bm25_custom_1st_stage_retrieval],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25","TF-IDF","Custom BM25"]
)

display(experiment_results)

# RUN THIS CODE IF GPU AND TIME AVAILABLE

MESSAGE: THE CODE BELOW IS SUPPOSED TO RETRIEVE THE WHOLE QUERIES BUT BECAUSE OF COMPUTATION SPEED, IT DIDN'T GIVE A RESULT IN A REASONABLE TIME (37mins)

In [ ]:

experiment_results = pt.Experiment(
    [bm25_1st_stage_retrieval, tf_idf_1st_stage_retrieval, bm25_custom_1st_stage_retrieval],
    queries_df, # Use all the queries
    qrels_df,   # Use all the qrels
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25","TF-IDF","BM25 Query Expansion histwords"]
)


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:449: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]


In [ ]:
display(experiment_results)
experiment_results.to_csv("experiment_results_all_queries.csv", index=False) # DON'T FORGET TO DOWNLOAD IT

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25,0.631272,0.5702,0.1426,0.07514,0.7130,0.7514,0.647859,0.660381
1,TF-IDF,0.631158,0.5704,0.1426,0.07516,0.7130,0.7516,0.647752,0.660316
2,BM25 Query Expansion histwords,0.630616,0.5687,0.1425,0.07528,0.7125,0.7528,0.647080,0.660198


# 2nd stage retrieval

In [18]:
# Initialize MonoT5ReRanker
monoT5_reranker = MonoT5ReRanker(batch_size=4)
monoT5_cleaned_reranker = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> monoT5_reranker
)

monoT5_raw_ocr_reranker = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={'raw_ocr': 'text'}))
    >> monoT5_reranker
)

query = "Europe"
monoT5_cleaned_results = monoT5_cleaned_reranker.search(query)
monoT5_raw_ocr_results = monoT5_raw_ocr_reranker.search(query)
print('-------------------- Cleaned MonoT5 Re-ranking results (top 5) --------------------\n', monoT5_cleaned_results.head(5))
print('-------------------- OCR MonoT5 Re-ranking results (top 5) --------------------\n', monoT5_raw_ocr_results.head(5))

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

-------------------- Cleaned MonoT5 Re-ranking results (top 5) --------------------
   qid   docid                     docno query_1 query_0   query  \
2   1    9637      Maryland_18401110_20  Europe  Europe  Europe   
3   1  110750      Nebraska_19020225_26  Europe  Europe  Europe   
0   1  122024      New_York_19190610_27  Europe  Europe  Europe   
4   1   36539  Rhode_Island_18511030_25  Europe  Europe  Europe   
1   1   61883         Texas_18771215_13  Europe  Europe  Europe   

                                                text     score  rank  
2  In the south it has almost accomplished its tr... -2.587005     0  
3  St. Louis, via Southampton (mail for Ireland m... -4.508429     1  
0  It should be a little full and extend about ei... -5.592597     2  
4  They said they pitied us, honored our efforts,... -5.927801     3  
1  When further we reflect that Russia is Slavoni... -6.968389     4  
-------------------- OCR MonoT5 Re-ranking results (top 5) --------------------
   qid

### LLM MODEL DEFINITION FOR QUESTION ANSWERING (GEMMA3 270M)

In [19]:
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
def gemma_qa_logic(df):
    prompts = [
        f"Context: {str(row['text'])[:1200]}\nQuestion: {row['query']}\n Short Answer:"
        for _, row in df.iterrows()
    ]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=False,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id
        )
    input_len = inputs.input_ids.shape[1]
    decoded_outputs = tokenizer.batch_decode(outputs[:, input_len:], skip_special_tokens=True)
    df["answer"] = [ans.strip().split('\n')[0] for ans in decoded_outputs]
    # print("---DEBUG---\n", df['answer'].value_counts())
    return df

==((====))==  Unsloth 2026.1.2: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


In [20]:
qa_context_pipeline = (
    query_pipeline
    >> bm25_custom_1st_stage_retrieval
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> monoT5_reranker
    >> pt.apply.generic(lambda df: df.sort_values(["qid", "score"], ascending=[True, False]).groupby("qid").head(3))
    >> pt.apply.generic(gemma_qa_logic)
)
qa_raw_ocr_pipeline = (
    query_pipeline
    >> bm25_custom_1st_stage_retrieval
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={'raw_ocr': 'text'}))
    >> monoT5_reranker
    >> pt.apply.generic(lambda df: df.sort_values(["qid", "score"], ascending=[True, False]).groupby("qid").head(3))
    >> pt.apply.generic(gemma_qa_logic)
)

### TESTING RERANKING IN A SUBSET OF THE QUERIES (20)

In [ ]:
queries_df_QA = pd.DataFrame(queries)
qrels_df_QA = pd.DataFrame(qrels)

queries_df_QA = queries_df_QA.rename(columns={"query_id": "qid", "question": "query"})
qrels_df_QA = qrels_df_QA.rename(columns={"query_id": "qid", "para_id": "docno"})

num_queries_subset_QA = 20
queries_subset_df_QA = queries_df_QA.head(num_queries_subset_QA)
qrels_subset_df_QA = qrels_df_QA[qrels_df_QA['qid'].isin(queries_subset_df_QA['qid'])].copy()

# Run the experiment with the QA re-ranking pipeline
experiment_results_QA = pt.Experiment(
    [monoT5_cleaned_reranker,monoT5_raw_ocr_reranker,qa_context_pipeline,qa_raw_ocr_pipeline],
    queries_subset_df_QA,  # Use the subset of queries
    qrels_subset_df_QA,    # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP], # Ensure these measures are defined
    names=["BM25_MonoT5_Reranked_Cleaned_Context", "BM25_MonoT5_Reranked_RawOCR","Custom RAG with context","Custom RAG with raw ocr"],
    precompute_prefix=True
)

display(experiment_results_QA)

# RUN THIS CODE IF GPU AND TIME AVAILABLE !

In [ ]:
import gc
import torch
import pandas as pd

if not pt.started():
    pt.init(mem=4000)

queries_df = pd.DataFrame(queries).rename(columns={"query_id": "qid", "question": "query"})
qrels_df = pd.DataFrame(qrels).rename(columns={"query_id": "qid", "para_id": "docno"})

queries_df['qid'] = queries_df['qid'].astype(str)
qrels_df['qid'] = qrels_df['qid'].astype(str)
qrels_df['docno'] = qrels_df['docno'].astype(str)

metrics = [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP]

configs = [
    ("BM25_MonoT5_Cleaned", monoT5_cleaned_reranker),
    ("BM25_MonoT5_RawOCR", monoT5_raw_ocr_reranker),
    ("Custom RAG Cleaned", qa_context_pipeline),
    ("Custom RAG Raw OCR", qa_raw_ocr_pipeline)
]

final_results = []

for name, pipeline in configs:
    print(f"\n>>> Starting Pipeline: {name}")
    res = pipeline.transform(queries_df)
    print(f"Evaluating {name}...")

    cols_to_keep = [c for c in ['qid', 'docno', 'score', 'rank'] if c in res.columns]
    res_for_eval = res[cols_to_keep].copy()

    eval_dict = pt.Utils.evaluate(res_for_eval, qrels_df, metrics=metrics)
    eval_dict["name"] = name
    final_results.append(eval_dict)

    del res
    del res_for_eval
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Done. Memory cleared.")

experiment_df = pd.DataFrame(final_results)
display(experiment_df)
experiment_df.to_csv("final_results_complete.csv", index=False)

/tmp/ipython-input-3097923738.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]



>>> Starting Pipeline: BM25_MonoT5_Cleaned


In [30]:
experiment_results_monoT5_E1 = pt.Experiment(
    [monoT5_cleaned_reranker,monoT5_raw_ocr_reranker,qa_context_pipeline,qa_raw_ocr_pipeline],
    queries_df,
    qrels_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_MonoT5_Reranked_Cleaned_Context", "BM25_MonoT5_Reranked_RawOCR","Custom RAG cleaned context","Custom RAG raw ocr"],
    precompute_prefix=True,
)

display(experiment_results_monoT5_E1)
experiment_results_monoT5_E1.to_csv("experiment_results_all_queries_2ndstage.csv", index=False)

/usr/local/lib/python3.12/dist-packages/pyterrier/_evaluation/_validation.py:43: UserWarning: Transformer Custom RAG cleaned context ((pt.apply.query() >> pt.apply.query() >> pt.apply.query() >> pt.apply.query() >> TerrierRetr(BM25) >> RankCutoff(10) >> <pyterrier.terrier._text_loader.TerrierTextLoader object at 0x7cfe27f9d340> >> pt.apply.generic() >> MonoT5(castorini/monot5-base-msmarco) >> pt.apply.generic() >> pt.apply.generic())) at position 2 failed to validate: Cannot determine outputs for (pt.apply.query() >> pt.apply.query() >> pt.apply.query() >> pt.apply.query() >> TerrierRetr(BM25) >> RankCutoff(10) >> <pyterrier.terrier._text_loader.TerrierTextLoader object at 0x7cfe27f9d340> >> pt.apply.generic() >> MonoT5(castorini/monot5-base-msmarco) >> pt.apply.generic() >> pt.apply.generic()) with inputs: ['qid', 'query'] - if your pipeline works, set validate='ignore' to remove this warning, or add transform_output method to the transformers in this pipeline to clarify how it works.

monoT5:   0%|          | 0/24996 [00:00<?, ?batches/s]

KeyboardInterrupt: 

In [29]:
queries_df = pd.DataFrame(queries).rename(columns={"query_id": "qid", "question": "query"})
qrels_df = pd.DataFrame(qrels).rename(columns={"query_id": "qid", "para_id": "docno"})

print("Running MonoT5 Cleaned...")
res_mono_cleaned = monoT5_cleaned_reranker.transform(queries_df)

print("Running MonoT5 Raw OCR...")
res_mono_raw = monoT5_raw_ocr_reranker.transform(queries_df)

print("Running RAG Cleaned (Gemma)...")
res_rag_cleaned = qa_context_pipeline.transform(queries_df)

print("Running RAG Raw OCR (Gemma)...")
res_rag_raw = qa_raw_ocr_pipeline.transform(queries_df)

# 3. Define metrics
metrics = [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP]

eval_list = []
results_map = {
    "BM25_MonoT5_Reranked_Cleaned_Context": res_mono_cleaned,
    "BM25_MonoT5_Reranked_RawOCR": res_mono_raw,
    "Custom RAG cleaned context": res_rag_cleaned,
    "Custom RAG raw ocr": res_rag_raw
}

for name, df in results_map.items():
    # evaluate() returns a dictionary of metrics for the dataframe
    e = pt.Utils.evaluate(df, qrels_df, metrics=metrics)
    e["name"] = name
    eval_list.append(e)

# 5. Compile and Display
experiment_results_all = pd.DataFrame(eval_list)
# Reorder columns to put 'name' first
cols = ['name'] + [c for c in experiment_results_all.columns if c != 'name']
experiment_results_all = experiment_results_all[cols]

display(experiment_results_all)
experiment_results_all.to_csv("experiment_results_all_queries_manual.csv", index=False)

Running MonoT5 Cleaned...


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


KeyboardInterrupt: 

### Using ASPIRE (TEMPORARY)


PyTerrier : https://pyterrier.readthedocs.io/en/latest/experiments.html

ASPIRE https://github.com/GiorgosPeikos/ASPIRE

ONLINE: https://aspire-ir-eval.streamlit.app/



Remember the res files are actually the rankings created by our retrieval model!

We downloading it to use another tool for data analysis. The tool is called ASPIRE.

ASPIRE requires a different input format!

In [ ]:
# ----- RESULTS (TF-IDF, BM25, etc.) -----
def save_res(df, run_name, out_path, sep="\t"):
    res = df.copy()  # keep original untouched
    if "docid" in res.columns and "docno" not in res.columns:
        res = res.rename(columns={"docid": "docno"})  # normalize doc id
    # The 'query' column is often added by pyterrier for inspection but not needed in output
    res = res.drop(columns=["query", "query_0"], errors="ignore") # remove query text, keep qid
    res["iteration"] = "Q0"  # fixed iteration value
    res["experiment_id"] = run_name  # label for the run
    res = res.rename(columns={"qid": "query_id", "docno": "doc_id"})  # final names
    # Ensure score is numeric for sorting, if needed, though pyterrier usually keeps it
    res["score"] = pd.to_numeric(res["score"])
    # Sort by score for ranking within each query_id
    res = res.sort_values(by=["query_id", "score"], ascending=[True, False])
    res["rank"] = res.groupby("query_id")["score"].rank(ascending=False, method="first").astype(int)
    res = res[["query_id", "iteration", "doc_id", "rank", "score", "experiment_id"]]  # correct column order
    res.to_csv(out_path, sep=sep, index=False, header=False)  # write file


# Generate full retrieval results for all queries
# Assuming 'bm25', 'tf_idf', 'bm25_hist_qe', 'queries_df' are available from previous cells
full_bm25_results = bm25.transform(queries_df)
full_tf_idf_results = tf_idf.transform(queries_df)
full_bm25_hist_qe_results = bm25_hist_qe.transform(queries_df)

# Save the results to CSV files for ASPIRE
save_res(full_bm25_results, "BM25", "results_bm25.csv")
save_res(full_tf_idf_results, "TF-IDF", "results_tf_idf.csv")
save_res(full_bm25_hist_qe_results, "BM25_HistQE", "results_bm25_hist_qe.csv")

print("Generated results CSVs for BM25, TF-IDF, and BM25 with historical QE.")

# ----- QUERIES -----
# root with task attribute
# Renaming 'qid' to 'query_id' for consistency if needed, but XML expects 'number' attribute
root = ET.Element("topics", attrib={"task": "Chronicling America QA"})  # create root element

# add one <topic number=\"...\"> per row
for _, row in queries_df.iterrows():  # iterate over dataframe rows (using queries_df)
    topic = ET.SubElement(root, "topic", attrib={"number": str(row["qid"])})  # topic node
    topic.text = str(row["query"])  # topic content

# pretty print
rough_xml = ET.tostring(root, encoding="utf-8")  # raw bytes
pretty_xml = minidom.parseString(rough_xml).toprettyxml(indent="  ", encoding="utf-8")  # formatted bytes

# write to file
with open("queries.xml", "wb") as f:  # save xml with declaration
    f.write(pretty_xml)

print("Generated queries.xml file.")

# ----- QRELS -----
qrels_out = qrels_df.copy() # Use qrels_df which is available in kernel state
qrels_out = qrels_out.drop(columns=["iteration"], errors="ignore") # iteration from qrels_df is int, ASPIRE wants string 'Q0'
if "docid" in qrels_out.columns and "docno" not in qrels_out.columns:
    qrels_out = qrels_out.rename(columns={"docid": "docno"})
qrels_out = qrels_out.rename(columns={"qid": "query_id", "docno": "doc_id"})
qrels_out["iteration"] = "0" # Ensure '0' string for ASPIRE format
qrels_out = qrels_out[["query_id", "iteration", "doc_id", "relevance"]]
qrels_out.to_csv("qrels.csv", sep=" ", index=False, header=False)

print("Generated qrels.csv file.")

print("All files generated for ASPIRE compatibility.")

In [ ]:
# To use ASPIRE, we need to download the res files, the qrels and queries in the
# required dataformat and use the interface or use it locally.

# Res file Expected format: query_id, iteration (i.e. Q0), doc_id, rank, score, experiment_id
# Queries Expected format: query_id, query
# Qrels Expected format: query_id, iteration (i.e. Q0), doc_id, relevance (without header row)

# DEMO TIME!